# Pythonでゴブレットゴブラーズ(簡易)のゲーム木を作成する

- [ゴブレットゴブラーズ (日本語版)](https://sgrk.blog.fc2.com/blog-entry-3687.html)
- [ソースコード設計スライド](https://docs.google.com/presentation/d/1K8fbRlk24Y_J3M--F-WnYYN0hddL89w3EGN4Zgw-L1g/edit?usp=sharing)

- [卒論概要](https://docs.google.com/document/d/1jEXvVmpUPCaowl3xR1qgyhUFGVHFVzyZjBo9F8OAR7M/edit?usp=sharing)

## 関数宣言

### 正規化

In [1]:
def convert_normalization_state(state):
  is_first = f'{state:060b}'[0]
  is_choise = f'{state:060b}'[1]
  is_choise_board = f'{state:060b}'[2]
  choise_piece_type = f'{state:060b}'[3:3+4]
  choise_piece = f'{state:060b}'[7:7+9]
  board = f'{state:060b}'[16:16+36]
  hand = f'{state:060b}'[52:52+8]

  # 変換表を定義
  convert_tables = [[6,3,0,7,4,1,8,5,2],[8,7,6,5,4,3,2,1,0],
                    [2,5,8,1,4,7,0,3,6],[2,1,0,5,4,3,8,7,6],
                    [6,7,8,3,4,5,0,1,2],[0,3,6,1,4,7,2,5,8],[8,5,2,7,4,1,6,3,0]]

  # 変換表でboardを変換したcand_boardを作成し、boardと比較して小さいなら上書きする
  # もしより小さい候補が見つかったら、それと同じ変換表でchoise_pieceも変換する
  board_list = list(board)
  choise_piece_list = list(choise_piece)
  for convert_table in convert_tables:
    cand_board_list = [0] * 36
    for i in range(4):
      for after_num, before_num in enumerate(convert_table):
        cand_board_list[i*9 + after_num] = board_list[i*9 + before_num]
    # print("cand_board : " + "".join(cand_board_list))  # デバッグ用
    # board_list と cand_board_list を比較
    if board_list > cand_board_list:
      board_list = cand_board_list # 上書き
      for after_num, before_num in enumerate(convert_table): # choise_pieceの変換
        choise_piece_list[after_num] = choise_piece_list[before_num]

  # board_list, choise_piece_listをboard, choise_pieceに変換
  board = "".join(board_list)
  choise_piece = "".join(choise_piece_list)

  normalization_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return normalization_state

### 合法手

In [2]:
def leagal_actions_from_choise_state(choise_state):
  is_choise_board = f'{choise_state:060b}'[2]
  choise_piece_type = f'{choise_state:060b}'[3:3+4]
  choise_piece = f'{choise_state:060b}'[7:7+9]
  board = f'{choise_state:060b}'[16:16+36]  

  # set_actions_bitを作成
  set_actions_bit = f'{0:036b}' # 36bit。すべての駒と位置が行動不能で初期化
  # choiseしている駒と同じ駒は行動できる。
  cpy_f = choise_piece_type.find('1')
  set_actions_bit = set_actions_bit[:9*cpy_f] + '111111111' + set_actions_bit[9*(cpy_f+1):]
  # choiseしている駒がboardの駒なら、そのマスに行動出来ない
  if is_choise_board == '1': 
    cp_f = choise_piece.find('1')
    set_actions_bit = set_actions_bit[:9*cpy_f+cp_f] + '0' + set_actions_bit[9*cpy_f+cp_f+1:]
  # choiseした駒以上の大きさの駒があるマスには行動できない。
  for mass in range(9):
    if board[mass] == '1' or board[mass+18] == "1": # L駒があるか
      set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]
    if cpy_f in [1,3]: #選択してあるこまがMなら
      if board[mass+9] == '1' or board[mass+18+9] == "1": # M駒があるか
        set_actions_bit = set_actions_bit[:9*cpy_f+mass] + '0' + set_actions_bit[9*cpy_f+mass+1:]

  # actions_bitをactionsに変換
  set_actions = []
  for i, bit in enumerate(set_actions_bit):
    if bit == '1':
      set_action = int('0b_1_0000_000000000', 2)
      set_action = int(f'{set_action:014b}'[:1+(i//9)] + '1' + f'{set_action:014b}'[1+(i//9)+1:], 2) # 駒の種類の反映
      set_action = int(f'{set_action:014b}'[:1+4+(i%9)] + '1' + f'{set_action:014b}'[1+4+(i%9)+1:], 2) # 駒の位置の反映
      set_actions.append(set_action)

  return set_actions

In [3]:
def leagal_actions_from_set_state(set_state):
  is_first = f'{set_state:060b}'[0]
  board = f'{set_state:060b}'[16:16+36]
  hand = f'{set_state:060b}'[52:52+8]

  # 盤面の各マスに駒が存在するかどうか
  exist_place = f'{int(board[0:9], 2) | int(board[9:18], 2) | int(board[18:27], 2) | int(board[27:36], 2) :09b}' 
  # 手駒は,
  choise_hand_actions_bit = '1111'
  for piece_type in range(4): # 手駒にその駒が１つもないなら行動不可
    if hand[piece_type*2:piece_type*2+2] == "00": 
      choise_hand_actions_bit = choise_hand_actions_bit[:piece_type] + '0' + choise_hand_actions_bit[piece_type+1:]
  if is_first == "1": # 先手なら、先手の駒しか動かせない
    choise_hand_actions_bit = choise_hand_actions_bit[:2] + '00'
  else: # 後手なら、後手の駒しか動かせない
    choise_hand_actions_bit = '00' + choise_hand_actions_bit[2:]
  # 盤上の駒は、
  exist_large_place = int(board[0:9], 2) | int(board[18:27], 2)
  exist_middle_place = int(board[9:18], 2) | int(board[27:36], 2)
  # 盤上の一番表面にある駒しか行動できない
  choise_board_actions_bit = board[0:9] \
   + f'{int(board[9:18], 2) & ~exist_large_place :09b}' \
   + board[18:27] \
   + f'{int(board[27:36], 2) & ~exist_large_place :09b}'
  if is_first == "1": # 先手なら、先手の駒しか動かせない
    choise_board_actions_bit = choise_board_actions_bit[:18] + '000000000000000000'
  else: # 後手なら、後手の駒しか動かせない
    choise_board_actions_bit = '000000000000000000' + choise_board_actions_bit[18:]

  # choise_hand_actions_bitをchoise_actionsへchoise_actionに変換しながら追加
  choise_actions = []
  for i, bit in enumerate(choise_hand_actions_bit): # choise_hand_actions_bit = '00_00
    if bit == '1':
      choise_action = int('0b_0_0000_000000000', 2)
      choise_action = int(f'{choise_action:014b}'[:1+i] + '1' + f'{choise_action:014b}'[1+i+1:], 2) # 駒の種類の反映
      choise_actions.append(choise_action)

  # choise_board_actions_bitをchoise_actionsへchoise_actionに変換しながら追加
  for i, bit in enumerate(choise_board_actions_bit): # choise_board_actions_bit = '000000000_000000000_000000000_000000000'
    if bit == '1':
      choise_action = int('0b_1_0000_000000000', 2)
      choise_action = int(f'{choise_action:014b}'[:1+(i//9)] + '1' + f'{choise_action:014b}'[1+(i//9)+1:], 2) # 駒の種類の反映
      choise_action = int(f'{choise_action:014b}'[:1+4+(i%9)] + '1' + f'{choise_action:014b}'[1+4+(i%9)+1:], 2) # 駒の位置の反映
      choise_actions.append(choise_action)

  return choise_actions

In [4]:
def legal_actions(state):
  actions = []
  if f'{state:060b}'[1] == '1':
    actions = leagal_actions_from_choise_state(state)
  else:
    actions = leagal_actions_from_set_state(state)
  return actions

### 次の手

In [5]:
def next_state_from_choise_state(choise_state, set_action):   
  is_first = f'{choise_state:060b}'[0]
  is_choise_board = f'{choise_state:060b}'[2]
  choise_piece_type = f'{choise_state:060b}'[3:3+4]
  choise_piece = f'{choise_state:060b}'[7:7+9]
  board = f'{choise_state:060b}'[16:16+36]
  hand = f'{choise_state:060b}'[52:52+8]

	# set_actionで選択している駒と位置を、boardで1にする。
  piece_type = f'{set_action:014b}'[1:5].find('1')
  piece_place = f'{set_action:014b}'[5:].find('1')
  board = board[:piece_type*9+piece_place] + '1' +  board[piece_type*9+piece_place+1:] 

	# 手番プレイヤーの交代 … is_firstを反転
  is_first = "0" if is_first == "1" else "1"
	# stateのモードを切り替える...is_choiseを0にする
  is_choise = "0"
  # is_choise_board, choise_piece_type, choise_piece をすべて0埋め
  is_choise_board = "0"
  choise_piece_type = "0000"
  choise_piece = "000000000"

  # next_stateの作成
  next_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return next_state

In [6]:
def next_state_from_set_state(set_state, choise_action):
  is_first = f'{set_state:060b}'[0]
  is_choise_board = f'{set_state:060b}'[2]
  choise_piece_type = f'{set_state:060b}'[3:3+4]
  choise_piece = f'{set_state:060b}'[7:7+9]
  board = f'{set_state:060b}'[16:16+36]
  hand = f'{set_state:060b}'[52:52+8]

	# stateのモードを切り替える...is_choiseを1にする
  is_choise = "1"
  # choise_actionをis_choise_board, choise_piece_type, choise_piece に反映する
  is_choise_board = f'{choise_action:014b}'[0]
  choise_piece_type = f'{choise_action:014b}'[1:5]
  choise_piece =  f'{choise_action:014b}'[5:14]
  # 選択した駒をboard, handから取り除く
  piece_type = f'{choise_action:014b}'[1:5].find('1')
  piece_place = f'{choise_action:014b}'[5:].find('1')
  if f'{choise_action:014b}'[0] == "1":   # choise_actionで盤の駒を選択しているなら
    # choise_actionで選択している駒と位置を、boardで0にする。
    board = board[:piece_type*9+piece_place] + '0' +  board[piece_type*9+piece_place+1:] 
  # choise_actionで選択している駒の数を、handで1減らす。
  # “00”...その駒は持ってない, “01”...一つ持ってる, “11”...２つ持ってる	
  if hand[piece_type*2:piece_type*2+2] == "11":
    hand = hand[:piece_type*2] + "01" + hand[piece_type*2+2:]
  else: # hand[piece_type*2:piece_type*2+2] == "01":
    hand = hand[:piece_type*2] + "00" + hand[piece_type*2+2:]

  # next_stateの作成
  next_state = int(is_first + is_choise + is_choise_board + choise_piece_type + choise_piece + board + hand, 2)
  return next_state

In [7]:
def create_next_state(state, action):
  is_choise = f'{state:060b}'[1]
  next_state = 0
  if is_choise == "1":
    next_state = next_state_from_choise_state(state, action)
  else:
    next_state = next_state_from_set_state(state, action)
  next_state = convert_normalization_state(next_state) # 下記確認ではコメントアウト
  return next_state

### 盤面から勝敗確定かチェック

In [145]:
# 勝敗の有無
def is_win(single_surface):
  s = single_surface
  # 横, 縦, 左斜め, 右斜めのラインを調べる
  check_lines = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6]]
  for check_line in check_lines:
    if s[check_line[0]] and s[check_line[1]] and s[check_line[2]]:
      return True;
  return False;

In [163]:
# 表面の駒だけbitが立つboardを作成する
def create_surface(board):
  # boardの変換
  board_list_list = [0] * 4
  for i in range(4):
    board_list_list[i] = list(board[i*9:i*9+9])
  board_surface = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
                   [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
  # M駒を反映させる
  board_surface[1] = [int(i_str) for i_str in board_list_list[1]]
  board_surface[3] = [int(i_str) for i_str in board_list_list[3]]

  # このとき、おいた場所により小さい駒があったら、その駒を0にする
  # １マスずつ見ていく
  for place in range(9):
    # L駒を反映させる
    if board_list_list[0][place] == "1" or board_list_list[2][place] == "1":
      if board_list_list[0][place] == "1":
        board_surface[0][place] = 1
      if board_list_list[2][place] == "1":
        board_surface[2][place] = 1
      board_surface[1][place] = 0 # M駒を0にする
      board_surface[3][place] = 0
  return board_surface

In [173]:
#勝敗の有無、勝者を確認する
def check_result(state):
  board = f'{state:060b}'[16:16+36]
  board_surface = create_surface(board)
  is_done = 0 # 決着がついているなら1を返す
  winner = 0 # 先手は0, 後手は1
  # 内部でboard_surface[0,1]とboard_surface[2,3]を合成
  single_surfaces = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]];
  for i in range(9):
    if board_surface[0][i] == 1 or board_surface[1][i] == 1:
      single_surfaces[0][i] = 1
    elif board_surface[2][i] == 1 or board_surface[3][i] == 1:
      single_surfaces[1][i] = 1
  if is_win(single_surfaces[0]) == 1 or  is_win(single_surfaces[1]) == 1:
    is_done = 1
  winner = 1 if is_win(single_surfaces[1]) == 1 else 0
  return [is_done, winner]

## 全状態の列挙

In [174]:
import time
from collections import deque
import sys
from IPython.display import clear_output

In [208]:
%%time

# 到達可能な全盤面を作成する
# 発見 -> 未訪問 -> 訪問済み の流れ。

q = deque() # 発見キュー
all_states = set() # 訪問済み

# 初期状態の追加
init_state = 0b_1_0_0_0000_000000000_000000000000000000000000000000000000_11111111
q.append(convert_normalization_state(init_state))
# qが空になるまで探索
cnt = 0
while q: 
  cnt += 1
  if cnt % 100 == 0:
    clear_output(wait=True)
    print(cnt)
    time.sleep(0.001) #1msの休止
  current_state = q.popleft()
  if current_state not in all_states: # current_stateがまだ訪問済みでないなら
    all_states.add(current_state) # 訪問済みに追加
    is_done, winner = check_result(current_state)
    if not is_done: # 勝敗決定盤面でないなら
      for action in legal_actions(current_state): # 合法手を列挙
        q.append(create_next_state(current_state, action)) #次の状態(正規化済)をqに追加
        
# list化してソート
allstates = list(all_states)
allstates.sort()

# 状態の確認
clear_output(wait=True)
print(f'状態数: {len(allstates)}')
print(f'サイズ: {sys.getsizeof(allstates)} byte')
print(f'loop回数: {cnt}')

状態数: 5116724
サイズ: 40933848 byte
loop回数: 21791191
Wall time: 36min 2s


In [209]:
# pickleで保存
import pickle
with open('allstates.pickle', 'wb') as f:
    pickle.dump(allstates, f)

## 後退解析

### 追加関数

In [204]:
# 勝敗判定を行う
# 勝ち色を返す (1...R, -1...B, 未確定なら0)
# 組み込んだ
def newWinLose(arg_allstates, arg_winLose, arg_state):
  is_first = int(f'{arg_state:060b}'[0])
  turn_color = 1 if is_first == 1 else -1
  alllose = True;
  for action in legal_actions(state): # 合法手を列挙
    next_state = create_next_state(arg_state, action)
    i1 = arg_allstates.index(next_state)
    if arg_winLose[i1] == turn_color: # 勝ち盤面が１つでもあればそれを指せば勝ち
      return turn_color;
    if arg_winLose[i1] == 0:
      alllose = False;
  if alllose: # 次の手がすべて相手の勝ち盤面なら相手の勝ち
    return -turn_color;
  else:
    return 0;

### 処理

In [205]:
# allstatesを非pickle化
with open('allstates.pickle', 'rb') as f:
    allstates = pickle.load(f)

# winLose, winLoseCountを全状態数分用意する
# winLose[i]... allstates[i]の勝敗. 未確定が0, R勝ちが1, B勝ちが-1
# winLoseCount ... allstates[i]が決着までかかる手数
winLose = [0] * len(allstates)  # その盤面の勝敗 0は未確定, 1はR, -1はB
winLoseCount = [0] * len(allstates) # 勝敗にかかるまでの手数
count= {"Redwin": 0, "Bluewin" : 0, "yet" : 0}
print(len(allstates))


44119


In [206]:
# 進行バー
from tqdm.notebook import tqdm as tqdm
for i in tqdm(range(100), desc="進捗", leave=True ):
  time.sleep(0.1)

In [207]:
%%time

# 現在の盤面のwinLoseを記録
for i, state in enumerate(tqdm(allstates[:], desc="初回の勝敗判定", leave=True)):
  is_done, winner = check_result(state)
  if is_done: # 終了盤面
    if winner == 0: #FIRST_PLAYER RED
      winLose[i]=1;
      count["Redwin"] += 1;
    elif winner == 1: #SECOND_PLAYER BLUE
      winLose[i]=-1;
      count["Bluewin"] += 1;
  else:
      winLose[i]=0;
      count["yet"] += 1;

print("後退解析開始")
c = 0 # 手数
while True:
  # 勝敗のカウント
  c += 1;
  print(f'iteration {c}')
  print(f'Redwin : {count["Redwin"]}, Bluewin : {count["Bluewin"]}, yet : {count["yet"]}')
  # 解析終了フラグ
  changed = False 
  # 解析
  for i, state in enumerate(tqdm(allstates[:1000], desc=f'iteration {c}', leave=True)):
    if winLose[i] == 0: # 未決着の盤面のみ判定する
      if 0:
        nv = newWinLose(allstates, winLose, state) # 勝敗
      if 1: # newWinLoseを組み込んだ
        is_first = int(f'{arg_state:060b}'[0])
        turn_color = 1 if is_first == 1 else -1
        alllose = True
        for action in legal_actions(state): # 合法手を列挙
          next_state = create_next_state(state, action)
          i1 = allstates.index(next_state)
          if winLose[i1] == turn_color: # 勝ち盤面が１つでもあればそれを指せば勝ち
            nv == turn_color;
            break;
          if winLose[i1] == 0:
            alllose = False;
        if nv == turn_color:
            pass;
        elif alllose: # 次の手がすべて相手の勝ち盤面なら相手の勝ち
          nv == -turn_color;
        else:
          nv == 0;
    
      if nv != 0:
        winLose[i] = nv # この状態の理論値を入力
        winLoseCount[i] = c # 勝敗にかかるまでの手数を入力
        if nv == 1:
          count["Redwin"] += 1;#決着の盤面数を増やす
        elif nv == -1:
          count["Bluewin"] += 1;
        count["yet"] -= 1; # 未決着盤面数をへらす
        changed = True # 解析継続
  if not changed:
    break;

print("解析終了")
print(f'Redwin : {count["Redwin"]}')
print(f'Bluewin : {count["Bluewin"]}')
print(f'あいこ : {count["yet"]}')
print(f'ループ回数 : {c}')


後退解析開始
iteration 1
Redwin : 693, Bluewin : 20, yet : 43406


ValueError: 306244774661211700 is not in list

In [189]:
# winLose, winLoseCountをpickle化
import pickle
with open('winLose.pickle', 'wb') as f:
    pickle.dump(winLose, f)
with open('winLoseCount.pickle', 'wb') as f:
    pickle.dump(winLoseCount, f)
